In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x782f9e57bec0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x782f9e5a4800>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi,I am Rahul I am a chied Ai engineer")])

AIMessage(content="Hi Rahul,\n\nIt's nice to meet you! That's a very impressive title.  \n\nWhat kind of AI work are you involved in? I'm always curious to learn more about what other AI engineers are up to.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 20, 'total_tokens': 72, 'completion_time': 0.094545455, 'prompt_time': 8.731e-05, 'queue_time': 0.02056543, 'total_time': 0.094632765}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6f6019c1-de55-4c97-8efd-abd376f5482f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 52, 'total_tokens': 72})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi,I am Rahul I am a chied Ai engineer"),
        AIMessage(content="Hi Rahul,\n\nIt's nice to meet you! That's a very impressive title.  \n\nWhat kind of AI work are you involved in? I'm always curious to learn more about what other AI engineers are up to.\n"),
        HumanMessage(content="Hey Whats my name and what do i do?")
    ]
)

AIMessage(content="You told me your name is Rahul, and that you are a chief AI engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  I'm happy to chat. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 90, 'total_tokens': 139, 'completion_time': 0.089090909, 'prompt_time': 0.00323649, 'queue_time': 0.020392018999999997, 'total_time': 0.092327399}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9e6529a0-770b-4d35-9740-d73f4717823f-0', usage_metadata={'input_tokens': 90, 'output_tokens': 49, 'total_tokens': 139})

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat_1"}}

In [17]:
response=with_message_history.invoke(
    [
        HumanMessage(content="Hi,I am Rahul I am a chied Ai engineer"),
    ],
    config=config
)

In [18]:
response.content

"Hi Rahul!\n\nIt's great to meet you. As a large language model, I'm always interested in connecting with other AI professionals. \n\nBeing a Chief AI Engineer is a fascinating role!  What kind of work do you specialize in? What are some of the most exciting projects you're working on right now? \n\nI'm eager to learn more about your work in the field. 😊  \n\n"

In [20]:
##change the config

config={"configurable":{"session_id":"chat_2"}}
response=with_message_history.invoke(
    [
        HumanMessage(content="Whats my name"),
    ],
    config=config
)

In [21]:
response.content

"As an AI, I have no memory of past conversations and no access to your personal information, including your name.\n\nIf you'd like to tell me your name, I'd be happy to know!\n"

In [33]:
## Prompt Template

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant,Answer all questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")

    ]
)
chain=prompt|model

In [32]:
chain.invoke({"messages":[HumanMessage(content="Hi,I am Rahul I am a chied Ai engineer")],"language":"Hindi"})

AIMessage(content='नमस्ते राहुल!  मुझे खुशी है कि आपसे मिली।  \n\nएक चीफ एआई इंजीनियर होने के नाते आप क्या काम कर रहे हैं? क्या मैं आपके किसी काम में मदद कर सकता हूँ? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 37, 'total_tokens': 96, 'completion_time': 0.107272727, 'prompt_time': 0.001706935, 'queue_time': 0.022400454, 'total_time': 0.108979662}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-34c1b3b1-4096-4f89-b9ea-fac94b6c7a00-0', usage_metadata={'input_tokens': 37, 'output_tokens': 59, 'total_tokens': 96})

In [37]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history
                                                ,input_messages_key="messages"
                                                )

In [41]:
config ={"configurable":{"session_id":"chat_4"}}
response=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="Hi I am Krish")],
        "language":"Hindi"
    },
    config=config
)
response.content

'नमस्ते कृष्ण!  😊  आप कैसे हैं? \n\n(Hello Krish!  😊  How are you?) \n'

In [44]:
response=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="What is my name?")],
        "language":"Hindi"
    },
    config=config
)
response.content

'आपका नाम कृष्ण है। 😊\n\n(Your name is Krish. 😊) \n'

In [45]:
## Managing the Conversation History

from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=7-,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="you are good asistant"),
    HumanMessage(content="Hi I am BOb")
]
# trimmer.invoke(messages)

/home/monopoly21/Desktop/projects/genai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[]

In [54]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {
        "messages" : messages + [HumanMessage(content="what ice cream do i like?")],
        "language":"English"
    }
)
response.content

'I will do my best to answer all your questions in English to the best of my ability!\n\nAsk away! 😊  \n\n'

In [ ]:
## Lets wrap this in message history



In [55]:
from langchain_core.documents import Document
documents=[
    Document(
        page_content="I Am Rahul",
        metadata={"source":"bird-doc"}
    )
]

In [56]:
documents

[Document(metadata={'source': 'bird-doc'}, page_content='I Am Rahul')]

In [65]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x782df8345ac0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x782df8347500>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [66]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") 

In [67]:
## VectorStore
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents,embedding=embeddings)



In [ ]:
### Retrievers
